# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kungsbacka,57.4872,12.0761,279.92,90,100,2.68,SE,1731269673
1,1,yatou,37.1500,122.3833,284.61,92,62,1.45,CN,1731269674
2,2,anadyr,64.7500,177.4833,268.43,86,75,3.00,RU,1731269675
3,3,olonkinbyen,70.9221,-8.7187,274.79,99,100,9.88,SJ,1731269677
4,4,utrik,11.2278,169.8474,301.48,76,0,9.30,MH,1731269678


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
#Function to convert Kelvin to Fahrenheit
def kelvin_convert(kelvin):
    return (kelvin - 273.15) * 9/5 +32

kelvins = city_data_df['Max Temp'].apply(kelvin_convert)

city_data_df.insert(5, "Max Temp (F)", kelvins, True)
city_data_df.rename(columns={"Max Temp": "Max Temp (K)"})

,City_ID,City,Lat,Lng,Max Temp (K),Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kungsbacka,57.4872,12.0761,279.92,44.186,90,100,2.68,SE,1731269673
1,1,yatou,37.1500,122.3833,284.61,52.628,92,62,1.45,CN,1731269674
2,2,anadyr,64.7500,177.4833,268.43,23.504,86,75,3.00,RU,1731269675
3,3,olonkinbyen,70.9221,-8.7187,274.79,34.952,99,100,9.88,SJ,1731269677
4,4,utrik,11.2278,169.8474,301.48,82.994,76,0,9.30,MH,1731269678
...,...,...,...,...,...,...,...,...,...,...,...
590,590,sechura,-5.5569,-80.8222,297.22,75.326,60,99,7.62,PE,1731270668
591,591,vilyuysk,63.7553,121.6247,260.74,9.662,96,100,2.23,RU,1731270669
592,592,shibganj,24.6829,88.1598,294.92,71.186,71,0,1.40,BD,1731270670
593,593,sergeyevka,53.8800,67.4158,263.79,15.152,96,71,2.93,KZ,1731270672


In [4]:
%%capture --no-display

# Configure the map plot

map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values

#Looking for max temps in my specified range
#Less than 85 and greater than 60
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp (F)'] < 85) & 
    (city_data_df['Max Temp (F)'] > 60) & 
    (city_data_df['Wind Speed'] < 6) &
    (city_data_df['Cloudiness'] >= 33) &
    (city_data_df['Humidity'] >= 20)
]
ideal_weather_df

# Drop any rows with null values
sample_weather_df = ideal_weather_df.dropna()

# Display sample data
sample_weather_df

,City_ID,City,Lat,Lng,Max Temp,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date
6,6,fale old settlement,-9.3852,-171.2468,300.92,81.986,77,98,5.50,TK,1731269680
10,10,praia da vitoria,38.7333,-27.0667,292.29,66.452,88,40,5.84,PT,1731269685
11,11,middelburg,-25.7751,29.4648,290.17,62.636,83,100,3.85,ZA,1731269687
16,16,kralendijk,12.1500,-68.2667,301.90,83.750,68,97,1.62,BQ,1731269693
25,25,mancio lima,-7.6142,-72.8958,301.19,82.472,83,75,1.03,BR,1731269704
...,...,...,...,...,...,...,...,...,...,...,...
581,581,port douglas,-16.4833,145.4667,299.03,78.584,78,90,5.76,AU,1731270656
584,584,marsabit,2.3284,37.9899,295.06,71.438,58,86,2.73,KE,1731270660
585,585,madang,-5.2246,145.7966,298.36,77.378,90,100,1.10,PG,1731270661
587,587,cockburn town,21.4612,-71.1419,301.13,82.364,89,75,4.12,TC,1731270664


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = sample_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
6,fale old settlement,TK,-9.3852,-171.2468,77,
10,praia da vitoria,PT,38.7333,-27.0667,88,
11,middelburg,ZA,-25.7751,29.4648,83,
16,kralendijk,BQ,12.1500,-68.2667,68,
25,mancio lima,BR,-7.6142,-72.8958,83,
...,...,...,...,...,...,...
581,port douglas,AU,-16.4833,145.4667,78,
584,marsabit,KE,2.3284,37.9899,58,
585,madang,PG,-5.2246,145.7966,90,
587,cockburn town,TC,21.4612,-71.1419,89,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000 
categories = "accommodation.hotel"
 
params = {
    'categories': categories,
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
fale old settlement - nearest hotel: No hotel found
praia da vitoria - nearest hotel: Salles
middelburg - nearest hotel: No hotel found
kralendijk - nearest hotel: Hotel Bonaire Inn
mancio lima - nearest hotel: No hotel found
kapa'a - nearest hotel: Pono Kai Resort
santo antonio do monte - nearest hotel: Hotel São Bento
arraial do cabo - nearest hotel: No hotel found
daru - nearest hotel: Kuki
masuguru - nearest hotel: No hotel found
menongue - nearest hotel: No hotel found
ardakan - nearest hotel: هتل رستوران باغ سرای اردکان
reggane - nearest hotel: No hotel found
weno - nearest hotel: High Tide Hotel
'alaqahdari dishu - nearest hotel: No hotel found
puerto deseado - nearest hotel: Los Olmos
winterville - nearest hotel: Country Inn & Suites, Greenville, NC
mar del plata - nearest hotel: Hotel 3 de Abril
zachary - nearest hotel: Holiday Inn Express & Suites Baton Rouge North
suva - nearest hotel: Extension 68
port shepstone - nearest hotel: The Spot Backpackers'
a

,City,Country,Lat,Lng,Humidity,Hotel Name
6,fale old settlement,TK,-9.3852,-171.2468,77,No hotel found
10,praia da vitoria,PT,38.7333,-27.0667,88,Salles
11,middelburg,ZA,-25.7751,29.4648,83,No hotel found
16,kralendijk,BQ,12.1500,-68.2667,68,Hotel Bonaire Inn
25,mancio lima,BR,-7.6142,-72.8958,83,No hotel found
...,...,...,...,...,...,...
581,port douglas,AU,-16.4833,145.4667,78,Mantra Aqueous Port Douglas
584,marsabit,KE,2.3284,37.9899,58,Chicho Hotel
585,madang,PG,-5.2246,145.7966,90,Madang Star International
587,cockburn town,TC,21.4612,-71.1419,89,The Salt Raker Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [33]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_plot

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)